In [ ]:
#-------------------------------------------
# Part III: Secure Device Boot Verification
#-------------------------------------------
import hashlib, binascii, machine, time
import os

# ============================================================
# LED Initialization + Security Configuration
# ============================================================
# Pin 2 is usually connected to the onboard LED on most ESP32 boards.
# MAX_TRIES controls how many wrong login attempts are allowed before lockout.
led = machine.Pin(2, machine.Pin.OUT)
MAX_TRIES = 3

# ============================================================
# Function: Generate SHA256 hash with Salt
# ============================================================
# password : user input password
# salt     : random 16 bytes added to strengthen hashing and prevent rainbow attacks
def hash_with_salt(password, salt):
    m = hashlib.sha256()
    m.update(salt + password.encode())   # combine salt + encoded password
    return binascii.hexlify(m.digest()).decode()   # return hash as readable hex text string

# ============================================================
# Check if "auth.txt" exists to know whether registration was done before
# ============================================================
# First boot: no stored password → create user credentials
if "auth.txt" not in os.listdir():

    print("\n🔹 First Boot — No registered user found")
    password = input("Set a new password: ")

    salt = os.urandom(16)  # 16 byte cryptographic random salt
    stored_hash = hash_with_salt(password, salt)

    # =====================================================
    # Save Salt + Hash to device internal flash
    # Data is stored in sequence: [salt (16 bytes)] + [hash (hex string)]
    # =====================================================
    with open("auth.txt", "wb") as f:
        f.write(salt + stored_hash.encode())

    print("\n✔ Account created and credentials saved in flash!")
    print("✨ You will not need to re-register after reboot.\n")

# ============================================================
# If "auth.txt" exists → device already has a password stored
# ============================================================
else:
    print("\n==== Device Login Interface ====")

    # Load stored authentication data
    with open("auth.txt", "rb") as f:
        data = f.read()
        salt = data[:16]           # first 16 bytes = stored salt
        stored_hash = data[16:].decode()  # remaining bytes = stored sha256 hash string

    # Login attempt system with failure counter
    tries = 0
    while tries < MAX_TRIES:
        login = input("Enter password: ")
        login_hash = hash_with_salt(login, salt)

        # =====================================================
        # Successful login → grant access, flash LED once
        # =====================================================
        if login_hash == stored_hash:
            print("\n✔ Access granted — system unlocked")
            led.value(1)
            time.sleep(4)
            led.value(0)

            # Here you can trigger secure system functions, boot scripts, etc.
            # Example: start sensors, enable WiFi, decrypt stored data...
            break

        else:
            tries += 1
            print(f"❌ Wrong password — {MAX_TRIES - tries} attempts remaining")

            # Warning LED blinking for wrong attempt
            for _ in range(2):
                led.value(1); time.sleep(0.3)
                led.value(0); time.sleep(0.3)

    # ============================================================
    # Lockout state — Too many failed attempts
    # ============================================================
    if tries == MAX_TRIES:
        print("\n🚨 Multiple failed attempts — Device locked for security!")

        # Permanent alert blinking (device stays blocked until reset)
        while True:
            led.value(1); time.sleep(0.2)
            led.value(0); time.sleep(0.2)




import hashlib, binascii, machine, time
import os

# ============================================================
# LED Initialization + Security Configuration
# ============================================================
# Pin 2 is usually connected to the onboard LED on most ESP32 boards.
# MAX_TRIES controls how many wrong login attempts are allowed before lockout.
led = machine.Pin(2, machine.Pin.OUT)
MAX_TRIES = 3

# ============================================================
# Function: Generate SHA256 hash with Salt
# ============================================================
# password : user input password
# salt     : random 16 bytes added to strengthen hashing and prevent rainbow attacks
def hash_with_salt(password, salt):
    m = hashlib.sha256()
    m.update(salt + password.encode())   # combine salt + encoded password
    return binascii.hexlify(m.digest()).decode()   # return hash as readable hex text string

# ============================================================
# Check if "auth.txt" exists to know whether registration was done before
# ============================================================
# First boot: no stored password → create user credentials
if "auth.txt" not in os.listdir():

    print("\n🔹 First Boot — No registered user found")
    password = input("Set a new password: ")

    salt = os.urandom(16)  # 16 byte cryptographic random salt
    stored_hash = hash_with_salt(password, salt)

    # =====================================================
    # Save Salt + Hash to device internal flash
    # Data is stored in sequence: [salt (16 bytes)] + [hash (hex string)]
    # =====================================================
    with open("auth.txt", "wb") as f:
        f.write(salt + stored_hash.encode())

    print("\n✔ Account created and credentials saved in flash!")
    print("✨ You will not need to re-register after reboot.\n")

# ============================================================
# If "auth.txt" exists → device already has a password stored
# ============================================================
else:
    print("\n==== Device Login Interface ====")

    # Load stored authentication data
    with open("auth.txt", "rb") as f:
        data = f.read()
        salt = data[:16]           # first 16 bytes = stored salt
        stored_hash = data[16:].decode()  # remaining bytes = stored sha256 hash string

    # Login attempt system with failure counter
    tries = 0
    while tries < MAX_TRIES:
        login = input("Enter password: ")
        login_hash = hash_with_salt(login, salt)

        # =====================================================
        # Successful login → grant access, flash LED once
        # =====================================================
        if login_hash == stored_hash:
            print("\n✔ Access granted — system unlocked")
            led.value(1)
            time.sleep(4)
            led.value(0)

            # Here you can trigger secure system functions, boot scripts, etc.
            # Example: start sensors, enable WiFi, decrypt stored data...
            break

        else:
            tries += 1
            print(f"❌ Wrong password — {MAX_TRIES - tries} attempts remaining")

            # Warning LED blinking for wrong attempt
            for _ in range(2):
                led.value(1); time.sleep(0.3)
                led.value(0); time.sleep(0.3)

    # ============================================================
    # Lockout state — Too many failed attempts
    # ============================================================
    if tries == MAX_TRIES:
        print("\n🚨 Multiple failed attempts — Device locked for security!")

        # Permanent alert blinking (device stays blocked until reset)
        while True:
            led.value(1); time.sleep(0.2)
            led.value(0); time.sleep(0.2)




